# PySpark RDD API
Ejercicios.

In [0]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 63kB/s 
     |████████████████████████████████| 204kB 46.0MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=9b5c6a7ae87df5146fb79fc8af5c89eb6661536d507375af9a7e08c50103c609
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
!pip install PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#idf = '1PN6TbhTwwl-hVII3P89IMbs1ashJ7BYz'
idf = '1QQ7k1Y5RvL0ZcnQuTVg0rHU6O5AK81zQ'
downloaded = drive.CreateFile({'id':idf})   # replace the id with id of file you want to access
downloaded.GetContentFile('flights.parquet') 
# Fuente del dataset: https://www.kaggle.com/divyansh22/flight-delay-prediction

In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
from pyspark.sql import SQLContext, SparkSession

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [0]:
sqlContext = SQLContext(sc)
rdd = sqlContext.read.parquet('flights.parquet').rdd.repartition(8).cache()

In [0]:
rdd.count()

598963

In [0]:
rdd.takeSample(False, 2)

In [0]:
# DISTANCE, DEP_TIME, ARR_TIME, DAY_OF_WEEK, ORIGIN, DEST, TAIL_NUM, DAY_OF_MONTH, DAY_OF_WEEK
# Ejemplo rdd.map(lambda x: (x.ORIGIN, x.DEST))

**Ejercicio 1:** Calcular la cantidad de vuelo por línea aérea (usar OP_UNIQUE_CARRIER). Calcular las diez (10) líneas aéreas con mayor cantidad de vuelos. Devolver una lista de Python con los códigos de estas 10 líneas.

**Ejercicio 2:** Calcular el promedio de vuelos que llegaron con 15 minutos de demora o mas (ARR_DEL15 ==1) para las 10 líneas con mas vuelos (usar el ejercicio anterior), de estas indicar las tres mejores y las tres peores.

**Ejercicio 3:** Calcular la cantidad de vuelos por ruta. Usando ORIGIN y DEST para la ruta. Devolver un rdd con la siguiente estructura: (RUTA, #Vuelos). Indicar además cuáles son las 10 rutas mas frecuentes y su cantidad de vuelos.

**Ejercicio 4:** Consideremos ahora la cantidad de líneas aéreas que transitan cada ruta, queremos saber cuáles son las diez rutas realizadas por mayor cantidad de líneas aéreas y cuáles son las diez líneas aéreas con mayor cantidad de rutas.
Devolver: Una lista de 10 tuplas de forma (ROUTE, #CARRIERS) y Una lista de 10 tuplas de tipo (Carrier, #ROUTES)

**Ejercicio 5:** Por cada ruta aérea calcular el promedio de tiempo de vuelo. Calculando ARR_TIME - DEP_TIME usando la función provista. Al calculo del tiempo de vuelo en minutos hay que sumarle TIMEDIFF que es la diferencia horaria entre las ciudades (en horas). Por lo tanto el calculo es:

```
hhmmtimediff(x.DEP_TIME,x.ARR_TIME) + (x.TIMEDIFF * 60)
```

Puntos extras: Además del promedio de tiempo de vuelo calcular la desviación standard del tiempo de vuelo para cada ruta.
Devolver: 
- Una lista de 10 tuplas de tipo (ROUTE, average_time)
- Una lista de 10 tuplas de tipo (ROUTE, time_std) (solo para las rutas con mas de 50 vuelos)

In [0]:
from numpy import sqrt
# Computes time diff in format HHMM (in minutes)
def hhmmtimediff(t1, t2):
  m2 = (t2 // 100) * 60 + (t2 % 100)
  m1 = (t1 // 100) * 60 + (t1 % 100)
  return m2 - m1 

**Ejercicio 6:** Para cada linea aerea contar cuantos vuelos tuvieron cuya duracion se excedio en 15 minutos o mas la duracion promedio de la ruta (para todas las lineas). Indicar las 10 mejores lineas aereas y las 10 peores de acuerdo a esta metrica.